In [1]:
import pandas as pd
import nltk
import string
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import model_selection, svm
from sklearn.metrics import accuracy_score

stopwords = nltk.corpus.stopwords.words('english')
lemmetizer = nltk.WordNetLemmatizer()

def remove_stop_words(text):
    text = [w for w in text if w.lower() not in stopwords]
    return text

def lemmetize_words(word_list):
    lemmetized = [lemmetizer.lemmatize(w) for w in word_list]
    return lemmetized

def remove_special_characters(text):
    text = [w for w in text if w.isalnum()]
    return text

def remove_non_english_words(text):
    printable = set(string.printable)
    return [word for word in text 
            if all(char in printable for char in word)]

def fix_text(row):
    return ' '.join(row)

df = pd.read_csv('amazon_cells_labelled.txt', sep="\t", header=None, engine='python')
df = df.rename(columns={0: 'text', 1: 'label'})
df = df.dropna()
df['final_text'] = df['text'].apply(nltk.word_tokenize)
df['final_text'] = df['final_text'].apply(remove_special_characters)
df['final_text'] = df['final_text'].apply(remove_non_english_words)
df['final_text'] = df['final_text'].apply(remove_stop_words)
df['final_text'] = df['final_text'].apply(lemmetize_words)
df['final_text'] = df['final_text'].apply(fix_text)

train_x, test_x, train_y, test_y = model_selection.train_test_split(df['final_text'], df['label'], test_size=0.2)

In [3]:
tfidvect = TfidfVectorizer(max_features=5000)
tfidvect.fit(df['final_text'])
train_x_tfidf = tfidvect.transform(train_x)
test_x_tfidf = tfidvect.transform(test_x)

In [4]:
model = svm.SVC()
model.fit(train_x_tfidf, train_y)
pred = model.predict(test_x_tfidf)
accuracy_score(pred, test_y)

0.79

In [7]:
import tweepy
import config
import re

consumer_key = config.api_key
consumer_secret = config.api_secrets
access_token = config.access_token
access_token_secret = config.access_secret
bearer_token = config.bearer_token

try:
    api = tweepy.Client(bearer_token=bearer_token)
except:
    print("Error: Authentication Failed")

def clean_text(text):
     return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])(\w+:\/\/\S+)", " ", text).split())   

tweets = tweepy.Paginator(api.search_recent_tweets , "Donald Trump", max_results = 100).flatten(limit=1000)
    
def count(list):
    positive_count = 0
    negative_count = 0
    for line in list:
        if model.predict(tfidvect.transform([clean_text(line.text)])) == 0:
            negative_count += 1
        elif model.predict(tfidvect.transform([clean_text(line.text)])) == 1:
            positive_count += 1

    return positive_count, negative_count

count(tweets)
#comment change

(168, 832)